In [ ]:
# Serialization and Deserialization
* create a persistent representation of the object
* transmit them to someone or something else outside our app

In [ ]:
# Pickling
* is a binary Serialization
* Security issues!

In [6]:
import pickle
ser = pickle.dumps('python string')
print(ser)
deser = pickle.loads(ser)
print(deser)
print(hex(id(ser)))
print(hex(id(deser)))
deser = pickle.loads(ser)
print(deser)

b'\x80\x03X\r\x00\x00\x00python stringq\x00.'
python string
0x1b5da0b8f80
0x1b5dc8a2230
python string


# JSON Serialization
* it is safe

In [14]:
import json
d1 = {'a':100, 'b':200}
d1_json = json.dumps(d1, indent=2)
print(type(d1_json))
print(d1_json)
d2 = json.loads(d1_json)
print(d1==d2)
print(d1 is d2)

<class 'str'>
{
  "a": 100,
  "b": 200
}
True
False


In [26]:
import json
from pprint import pprint
d_json = '''
{
    "name": "John Cardona",
    "age": 29,
    "heigth": 1.87,
    "walksFunny": true,
    "sketches": [
        {
            "title": "dead Parrot",
            "costarts": ["michael Palin"]
        },
        {
            "title": "ministry of silly walks",
            "costarts": ["michael Palin","Terry Jones"]
        }
    ],
    "boring": null
}
 '''
d = json.loads(d_json)
pprint(d)

{'age': 29,
 'boring': None,
 'heigth': 1.87,
 'name': 'John Cardona',
 'sketches': [{'costarts': ['michael Palin'], 'title': 'dead Parrot'},
              {'costarts': ['michael Palin', 'Terry Jones'],
               'title': 'ministry of silly walks'}],
 'walksFunny': True}


## Tuples are'nt recognized by json, tuples are automatically converted into lists 

In [28]:
import json
d = {'a':(1,2,3)}
ser = json.dumps(d)
deser = json.loads(ser)
print(deser)

{'a': [1, 2, 3]}


# Exceptions
* Decimal is not json serializabel
* complex is not json serializabel

# serialize Objects

In [37]:
import json
from pprint import pprint 
class Person:
    def __init__(self,name,age):
        self.name = name
        self.age = age
    def __repr__(self):
        return f'Person(name={self.name},age={self.age}'
    def toJSON(self):
        return dict(name=self.name, age=self.age)

p = Person('john',29)
p_json = json.dumps({'john' : p.toJSON()})
pprint(json.loads(p_json))


{'john': {'age': 29, 'name': 'john'}}


# Custom json Encoding

# Datetime serialization
* ISO 8601 fromat: YYYY-MM-DDTHH:MM:SS.mmmm

In [40]:
from datetime import datetime
import json
current = datetime.utcnow()
print(current)
log_record = {'time':datetime.utcnow().isoformat(), 'message':'testing'}
d_json = json.dumps(log_record)
pprint(d_json)

2021-05-07 21:13:00.810243
'{"time": "2021-05-07T21:13:00.810243", "message": "testing"}'


In [43]:
from datetime import datetime
import json

def custom_json_fromatter(arg):
    if isinstance(arg,datetime):
        return arg.isoformat()
    elif isinstance(arg,set):
        return list(arg)


current = datetime.utcnow()
log_record = {'time':current, 'message':'testing'}
d_json = json.dumps(log_record, default=custom_json_fromatter)
pprint(d_json)


'{"time": "2021-05-07T21:42:21.534914", "message": "testing"}'


In [51]:
from datetime import datetime
import json

class Person:
    def __init__(self,name,age):
        self.name = name
        self.age = age
        self.create_dt = datetime.utcnow()
    def __repr__(self):
        return f'Person(name={self.name}, age={self.age})'
    def toJSON(self):
        return {
            'name':self.name,
            'age':self.age,
            'create_dt':self.create_dt.isoformat()
        }

def custom_json_formatter(arg):
    if isinstance(arg,datetime):
        return arg.isoformat()
    elif isinstance(arg,set):
        return list(arg)
    elif isinstance(arg,Person):
        return arg.toJSON()

p = Person('John',82)
print(p)
pprint(p.toJSON())
print('\n')
log_record =dict(time=datetime.utcnow(),
                    message='Created new person record.',
                    person=p
                    )

d_json = json.dumps(log_record, default=custom_json_formatter)
pprint(d_json)

Person(name=John, age=82)
{'age': 82, 'create_dt': '2021-05-07T21:50:25.930114', 'name': 'John'}


('{"time": "2021-05-07T21:50:25.931121", "message": "Created new person '
 'record.", "person": {"name": "John", "age": 82, "create_dt": '
 '"2021-05-07T21:50:25.930114"}}')


# JSONENCODER CLASS

In [56]:
import json
from datetime import datetime

class CustomJSONEncoder(json.JSONEncoder):
    def default(self,arg):
        if isinstance(arg,datetime):
            return arg.isoformat()
        else:
            super().default(arg)

print(json.dumps(dict(name='test', time=datetime.utcnow()), cls=CustomJSONEncoder))

d = {
    'a':float('inf'),
    'b':float('nan')
}
ser = json.dumps(d)
print(ser)
deser = json.loads(ser)
print(deser)
print('Infinity and NAN are invalid Json')


{"name": "test", "time": "2021-05-08T01:15:37.726266"}
{"a": Infinity, "b": NaN}
{'a': inf, 'b': nan}


# use allow_nan=False to prevent invalid JSON values
# use skipkeys=True to skip invalid JSON values

# deal with custom separators

In [61]:
import json
class CustomEncoder(json.JSONEncoder):
    def __init__(self, *args, **kwargs):
        print(kwargs)
        super().__init__(skipkeys=kwargs['skipkeys'],
                        allow_nan=False,
                        indent='---',
                        separators=(';',('='))
                         )
    def default(self,arg):
        if isinstance(arg,datetime):
            return arg.isoformat()
        else:
            return super().default(arg)
d={
    'time':datetime.utcnow(),
    1+1j:"complex",
    'name':'python'
}

ser = json.dumps(d,cls=CustomEncoder,skipkeys=True)
print(ser)

{'skipkeys': True, 'ensure_ascii': True, 'check_circular': True, 'allow_nan': True, 'indent': None, 'separators': None, 'default': None, 'sort_keys': False}
{
---"time"="2021-05-08T01:33:44.079787";
---"name"="python"
}


# Custom JSON DECODING

In [6]:
import json
from pprint import pprint
from datetime import datetime

j = """ 
    {
        "time": {
            "objecttype": "datetime",
            "value": "2021-04-01T09:14:00"
        },
        "message": "created this json string"
    }
 """
d = json.loads(j)
for key, value in d.items():
    if (isinstance(value,dict) and
        'objecttype' in value and
        value['objecttype'] == 'datetime'
        ):
        d[key] = datetime.strptime(value['value'], '%Y-%m-%dT%H:%M:%S')
pprint(d)

{'message': 'created this json string',
 'time': datetime.datetime(2021, 4, 1, 9, 14)}


In [13]:
import json
from pprint import pprint
from datetime import datetime
from fractions import  Fraction

def custom_decoder(arg):
    if 'objecttype' in arg:
        if arg['objecttype'] == 'datetime':
            return datetime.strptime(arg['value'], '%Y-%m-%dT%H:%M:%S')
        elif arg['objecttype'] == 'fraction':
            return Fraction(arg['numerator'], arg['denominator'])
    return arg

j = """ 
    {
        "times": {
            "created": {
                "objecttype": "datetime",
                "value": "2021-04-01T09:14:00"
                },
            "updated": {
                "objecttype": "datetime",
                "value": "2021-05-02T10:15:00"
                }
        },
        "myShare": {
            "objecttype": "fraction",
            "numerator": 1,
            "denominator": 8 
        },
        "message": "created this json string"
    }
 """

d = json.loads(j, object_hook=custom_decoder)
pprint(d)

{'message': 'created this json string',
 'myShare': Fraction(1, 8),
 'times': {'created': datetime.datetime(2021, 4, 1, 9, 14),
           'updated': datetime.datetime(2021, 5, 2, 10, 15)}}


In [24]:
import json
from pprint import pprint
from datetime import datetime
from fractions import  Fraction

class Person:
    def __init__(self,name,ssn):
        self.name = name
        self.ssn = ssn
    def __repr__(self):
        return f'Person(name={self.name}, ssn={self.ssn})'
    def toJSON(self):
        return dict(objecttype='person', name=self.name, ssn=self.ssn)

def custom_decoder(arg):
    if 'objecttype' in arg:
        if arg['objecttype'] == 'datetime':
            return datetime.strptime(arg['value'], '%Y-%m-%dT%H:%M:%S')
        elif arg['objecttype'] == 'fraction':
            return Fraction(arg['numerator'], arg['denominator'])
        elif arg['objecttype'] == 'person':
            return Person(arg['name'], arg['ssn'])
        return arg
    return arg
j = '''
    {
        "accountHolder": {
            "objecttype": "person",
            "name": "Eric Idle",
            "ssn": 100
        },
        "created": {
            "objecttype": "datetime",
            "value": "2021-04-01T09:14:00"
        }
    }
'''

d = json.loads(j, object_hook=custom_decoder)
pprint(d)

{'accountHolder': Person(name=Eric Idle, ssn=100),
 'created': datetime.datetime(2021, 4, 1, 9, 14)}


# Parsing

In [36]:
from decimal import Decimal 
def myparse_decimal(arg):
    return Decimal(arg)
def myparse_int_binary(arg):
    return bin(int(arg))
def myparse_none(arg):
    return None

j=""" 
    {
        "a": 100,
        "b": 0.2,
        "c": 0.4,
        "d": null,
        "e": Infinity,
        "f": NaN
    }
 """
d = json.loads(j, parse_int=myparse_int_binary, parse_float=myparse_decimal,parse_constant=myparse_none)
pprint(d) 
print(help(json.loads))

{'a': '0b1100100',
 'b': Decimal('0.2'),
 'c': Decimal('0.4'),
 'd': None,
 'e': None,
 'f': None}
Help on function loads in module json:

loads(s, *, encoding=None, cls=None, object_hook=None, parse_float=None, parse_int=None, parse_constant=None, object_pairs_hook=None, **kw)
    Deserialize ``s`` (a ``str``, ``bytes`` or ``bytearray`` instance
    containing a JSON document) to a Python object.
    
    ``object_hook`` is an optional function that will be called with the
    result of any object literal decode (a ``dict``). The return value of
    ``object_hook`` will be used instead of the ``dict``. This feature
    can be used to implement custom decoders (e.g. JSON-RPC class hinting).
    
    ``object_pairs_hook`` is an optional function that will be called with the
    result of any object literal decoded with an ordered list of pairs.  The
    return value of ``object_pairs_hook`` will be used instead of the ``dict``.
    This feature can be used to implement custom decoders. 

# JSONDecoder

In [41]:
import json
from pprint import pprint
from datetime import datetime
from fractions import  Fraction
from decimal import  Decimal

class CustomDecoder(json.JSONDecoder):
    def decode(self,arg):
        print("decode: ",type(arg),arg)
        return "a simple string object"

j=""" 
    {
        "a": 100,
        "b": [1,2,3],
        "c": "python",
        "d": {
            "e": 4,
            "f": 5.5
            }
    }
 """

d = json.loads(j, cls=CustomDecoder)
print('-------------\n')
print(d)

decode:  <class 'str'>  
    {
        "a": 100,
        "b": [1,2,3],
        "c": "python",
        "d": {
            "e": 4,
            "f": 5.5
            }
    }
 
-------------

a simple string object


In [49]:
import json
from pprint import pprint
from datetime import datetime
from fractions import  Fraction
from decimal import  Decimal

class Point:
    def __init__(self,x,y):
        self.x=x
        self.y=y
    def __repr__(self):
        return f'Point(x={self.x},y={self.y})'

class CustomDecoder(json.JSONDecoder):
    def decode(self,arg):
        obj = json.loads(arg)
        if 'points' in obj:
            obj['points'] = [Point(x,y)
                            for x,y in obj['points']]
        return obj

j_points=""" 
    {
        "points": [
            [1,2],
            [1,0],
            [-1.2,-2.5]
        ]
    }
"""
j_others=""" 
    {
        "a": 1,
        "b": 2
    }
"""
d = json.loads(j_points, cls=CustomDecoder)
pprint(d)
print('\n')
d = json.loads(j_others, cls=CustomDecoder)
pprint(d)

{'points': [Point(x=1,y=2), Point(x=1,y=0), Point(x=-1.2,y=-2.5)]}


{'a': 1, 'b': 2}


# JSON Schemas

In [51]:
from jsonschema import validate
from jsonschema.exceptions import ValidationError
from json import loads, dumps, JSONDecodeError

